In [1]:
import pandas as pd
import numpy as np
import os
import gget
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import sys
import umap
import scanpy as sc
import itertools
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
import scipy

import utils as ut

2023-06-12 14:44:44.482686: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 14:44:50.035841: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 14:44:51.823409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-12 14:44:51.823434: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
cardDir =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"
xyPath =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/combinedEmbedding.csv"
xy = pd.read_csv(xyPath)

macTypes = [
    'Mac1',
    'Mac2',
    'Mac3',
    'Mac4',
    'Mac5',
]

print()

keys = ['ND', 'HFD8', 'HFD14']

rna = {}
labels = {}

for key in keys:
    print(f"{key}...")
    labelPath = f"{cardDir}{key}_macrophage_clusters.csv"
    lf = pd.read_csv(labelPath)
    
    lf = lf.drop_duplicates(subset='cellId')
    lf = lf[lf['cellType'].isin(macTypes)]
    
    lf = pd.merge(lf, xy[['x', 'y', 'cellId', 'colors']], 
                  how='left',
                  left_on='cellId',
                  right_on='cellId')
    

    cellIds = lf['cellId'].to_list()
    labels[key] = lf
    
    rnaPath = f"{cardDir}{key}_macrophage_rna.csv"
    rf = pd.read_csv(rnaPath)
    rf = rf.T
    new_header = rf.iloc[0] 
    rf = rf[1:] 
    rf.columns = new_header 
    rf.index.names = ['cellId']

    rf = rf[rf.index.isin(cellIds)]
    print(f"{key=} {lf.shape=} {rf.shape}")
    
    rf = ut.normalize(rf, 1e6)
    rna[key] = rf
    

print('done')


ND...
key='ND' lf.shape=(360, 6) (359, 31053)
HFD8...
key='HFD8' lf.shape=(2194, 6) (2188, 31053)
HFD14...
key='HFD14' lf.shape=(3893, 6) (3884, 31053)
done


In [3]:
# merge all expression
rdf = []
for key in keys:
    rf = rna[key].copy()
    rf.index = rf.index + "_" + key
    rdf.append(rf)

rdf = pd.concat(rdf)
print(f"{rdf.shape=}")
rdf.head()

rdf.shape=(6431, 31053)


Unnamed: 0,XKR4,GM1992,GM37381,RP1,SOX17,GM37323,MRPL15,LYPLA1,GM37988,TCEA1,...,AC125149.1,AC125149.2,AC125149.4,AC234645.1,AC168977.2,AC168977.1,AC149090.1,CAAA01118383.1,VMN2R122,CAAA01147332.1
cellId,,,,,,,,,,,,,,,,,,,,,
AAAGAACGTCTACAGT.1_ND,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,365.89828,...,0.0,0.0,0.0,0.0,0.0,0.0,365.89828,365.89828,0.0,0.0
AAAGAACTCCTCGCAT.1_ND,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1103.509159,220.701832,0.0,0.0
AAAGGGCGTTAGGGTG.1_ND,0.0,0.0,0.0,0.0,0.0,0.0,111.994624,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.994624
AACAAAGCACCGGCTA.1_ND,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,85.697146,0.0,0.0,0.0
AACAAAGGTTCACGAT.1_ND,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331.619964,0.0,331.619964,...,0.0,0.0,0.0,0.0,0.0,0.0,331.619964,331.619964,0.0,0.0


In [4]:
""" Build the reference dataframe """
ref = []
for key in keys:
    lf = labels[key]
    ref.append(lf)

ref = pd.concat(ref, ignore_index=True)
ref = ref.rename(columns={'sampleInfo' : 'key'})
ref['id'] = ref['cellId'] + "_" + ref['key']
print(f"{ref.shape=}")
ref.head()

ref.shape=(6447, 7)


,cellId,cellType,key,x,y,colors,id
0,AAAGAACGTCTACAGT.1,Mac1,ND,3.089298,-1.726030,#fdbf6f,AAAGAACGTCTACAGT.1_ND
1,AAAGAACTCCTCGCAT.1,Mac1,ND,3.614130,-1.571004,#fdbf6f,AAAGAACTCCTCGCAT.1_ND
2,AAAGGGCGTTAGGGTG.1,Mac3,ND,3.086352,6.166065,#fdbf6f,AAAGGGCGTTAGGGTG.1_ND
3,AACAAAGCACCGGCTA.1,Mac3,ND,4.344001,5.801922,#fdbf6f,AACAAAGCACCGGCTA.1_ND
4,AACAAAGGTTCACGAT.1,Mac1,ND,0.130051,-1.583494,#fdbf6f,AACAAAGGTTCACGAT.1_ND


In [5]:
ctypes = [
    'Mac1',
    'Mac2',
    'Mac3',
    'Mac4', 
    'Mac5',
]

cids = ref[ref['cellType'].isin(ctypes)]['id'].to_list()
pdf = rdf[rdf.index.isin(cids)].copy()

pdf = pd.merge(pdf, ref[['id', 'cellType']],
               how='left',
               left_index=True,
               right_on='id')


print(f"{pdf.shape=}")

pdf.shape=(6447, 31055)


In [6]:
"""GET GENE LIST"""

genes = [
    'Cd80',
    'Cxcl9',
    'Cxcl10',
    'Ccl22',
    'Cd86',
    'H2-Eb1',
    'H2-Eb2',
    'Ccl6',
    'Ccl9',
]

genes = [x.upper() for x in genes if x.upper() in pdf.columns]
print(len(genes))

9


In [ ]:
res = []

for ct in ctypes:
    print(f"working {ct}...")
    mask = pdf['cellType'] == ct
    inDf = pdf[mask]
    outDf = pdf[~mask]

    for g in genes:
        score, pval = scipy.stats.ranksums(inDf[g], outDf[g],
                                           alternative='two-sided')

        lfc = np.log2(inDf[g].mean()+1) - np.log2(outDf[g].mean()+1)

        row = {
            'gene' : g,
            "cellType" : ct,
            'score' : score,
            'pval' : pval,
            'log2foldchange' : lfc,
        }
        res.append(row)

res = pd.DataFrame(res)
res.head()

working Mac1...
working Mac2...


In [ ]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = 6, 2


pdx = res.copy()
pdx = pd.pivot_table(pdx, 
                     index='cellType',
                     columns='gene',
                     values='log2foldchange')

pdx.columns = [x.lower().capitalize() for x in pdx.columns]

sns.heatmap(data=pdx, 
            cmap='RdYlGn',
            lw=0.5,
            linecolor='k')

plt.yticks(rotation=0)
plt.ylabel("")
# plt.xlabel("Splicing Factor")

In [ ]:
# get macspectrum data 

# load the macspectrum data
macDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/macSpectrum/macspectrumData/GSE117176_RAW/"
min_genes = 100
min_cells = 10

msData = {}

for mkey in ["M0", 'M1', 'M2']:
    mtxPath = f"{macDir}{mkey}"
    
    data = sc.read_10x_mtx(mtxPath, 
                           # var_names='gene_symbols', 
                           cache=True)
    
    data.var_names = [x.upper() for x in data.var_names] # uppercase gene names
    sc.pp.filter_cells(data, min_genes=min_genes)
    sc.pp.filter_genes(data, min_cells=min_cells)
    sc.pp.normalize_total(data, target_sum=1e6) 
    
    df = data.to_df()
    df = df.loc[:,~df.columns.duplicated()].copy() # drop duplicate column names
    msData[mkey] = df
    print(mkey, df.shape)

print('done')

In [ ]:
# get genes expressed in all data sets
allGenes = rdf.columns.to_list()

for mkey in ["M0", 'M1', 'M2']:
    allGenes += msData[mkey].columns.to_list()

print(f"{len(allGenes)=}")
counts = Counter(allGenes)

allGenes = [x for x, count in counts.items() if count == 4]
print(f"{len(allGenes)=}")

In [ ]:
# Build the  macspectrum data frame

mdf = []
for mkey in ["M0", 'M1', 'M2']:
    mf = msData[mkey][allGenes].reset_index(drop=True)
    mf['cellType'] = mkey
    mdf.append(mf)

mdf = pd.concat(mdf)
print(f"{mdf.shape=}")

mdf.head()

# macspecturm DEGS

In [ ]:
# get macspectrum DEGs

deg = []

for ct in ["M0", 'M1', 'M2']:
    print(f"working {ct}...")
    mask = mdf['cellType'] == ct
    inDf = mdf[mask]
    outDf = mdf[~mask]

    for g in allGenes:
        score, pval = scipy.stats.ranksums(inDf[g], outDf[g],
                                           alternative='two-sided')

        lfc = np.log2(inDf[g].mean()+1) - np.log2(outDf[g].mean()+1)

        row = {
            'gene' : g,
            "cellType" : ct,
            'score' : score,
            'pval' : pval,
            'log2foldchange' : lfc,
        }
        deg.append(row)

deg = pd.DataFrame(deg)
deg.head()

In [ ]:
alpha = 0.05
ntests = len(deg) / 3
alphaHat = 1 - ((1-alpha) ** (1/ntests))
print(f"{alpha=} {ntests=} {alphaHat}")

sig = deg[deg['pval'] <= alphaHat]
print(f"{sig.shape=}")
print(sig['cellType'].value_counts())

print()
print(sig.head())

msTypeMap = dict(zip(sig['gene'], sig['cellType']))

In [ ]:
res = []

for ct in ctypes:
    print(f"working {ct}...")
    mask = pdf['cellType'] == ct
    inDf = pdf[mask]
    outDf = pdf[~mask]

    for g in sig['gene'].unique():
        score, pval = scipy.stats.ranksums(inDf[g], outDf[g],
                                           alternative='two-sided')

        lfc = np.log2(inDf[g].mean()+1) - np.log2(outDf[g].mean()+1)

        row = {
            'gene' : g,
            "cellType" : ct,
            'msType' : msTypeMap[g],
            'score' : score,
            'pval' : pval,
            'log2foldchange' : lfc,
        }
        res.append(row)
    
res = pd.DataFrame(res)
res.head()

In [ ]:
# plt.rcParams['figure.dpi'] = 300
# plt.rcParams['figure.figsize'] = 15, 3

# sigSorted = sig.sort_values(by='cellType')
# geneOrder = sigSorted['gene'].to_list()

# pdx = res.copy()
# pdx = pd.pivot_table(pdx, 
#                      index='cellType',
#                      columns='gene',
#                      values='log2foldchange')

# pdx = pdx[geneOrder]
# sns.heatmap(data=pdx, 
#             cmap='RdYlGn')

# plt.yticks(rotation=0)
# plt.ylabel("")
# # plt.xlabel("Splicing Factor")

In [ ]:
pdx = res.copy()

# remove non-significant genes
lfc = 1
alpha = 0.05
ntests = len(deg) / 5
alphaHat = 1 - ((1-alpha) ** (1/ntests))
print(f"{alpha=} {ntests=} {alphaHat}")

sigGenes = pdx[pdx['pval'] <= alphaHat]['gene'].to_list()

top = 400
sig = sig.groupby('cellType').head(top).reset_index(drop=True)


"""group by the msType to get the same number of genes per category"""

pdx = pdx[pdx['msType'].isin(['M1', 'M2'])]

pdx = pd.pivot_table(pdx, 
                     index='cellType',
                     columns='gene',
                     values='log2foldchange')



msColorMap = {
    'M0' : 'C0',
    'M1' : 'r',
    'M2' : 'g',
}

sigSorted = res.sort_values(by=['msType', 'log2foldchange'])
geneOrder = sigSorted['gene'].to_list()
geneOrder = [x for x in geneOrder if x in sigGenes]
geneOrder = [x for x in geneOrder if x in pdx.columns]

sigSorted['Activation'] = sigSorted['msType'].map(msColorMap)

column_colors = sigSorted[['gene', 'Activation']].drop_duplicates()
column_colors = column_colors.set_index('gene')

column_colors.head()

pdx = pdx[geneOrder]

pdx.head()


g = sns.clustermap(pdx, 
                   vmin=-2, 
                   vmax=2,
                   cmap='RdYlGn',
                   method='single',
                   metric='cosine',
                   # col_cluster=False,
                   row_cluster=False,
                   z_score=0,
                   figsize=(8, 2.5),
                   row_colors=ut.MACCOLORS,
                   col_colors=column_colors)


# g.cax.set_visible(False)
g.ax_heatmap.set_ylabel("")
g.ax_heatmap.set_xticks([])
g.ax_heatmap.set_yticks([])
g.ax_heatmap.set_xlabel("")

In [ ]:
sig.head()